In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [129]:
eventsData = pd.read_csv("data/Eventos(+completo).csv",low_memory = False)

In [130]:
trainingSet = pd.read_csv("data/labels_training_set.csv", low_memory = False)

In [131]:
eventsData.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,Parte del dia
0,0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,samsung galaxy j7 prime,Excelente,32GB,Dourado,...,NaN,NaN,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,Noche
1,1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iphone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,Noche
2,2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iphone 6s,Muito Bom,64GB,Prateado,...,NaN,NaN,Niterói,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 61.0,Noche
3,3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iphone 7,Bom,128GB,Vermelho,...,NaN,NaN,Juquitiba,Sao Paulo,Brazil,Computer,1360x768,Windows 8.1,Chrome 66.0,Noche
4,4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,lg g4 h818p,Excelente,32GB,Branco,...,NaN,NaN,Osasco,Sao Paulo,Brazil,Computer,1536x864,Windows 7,Chrome 66.0,Noche


In [132]:
trainingSet.head()


,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [133]:
geoData = eventsData[["person","region","city"]]

In [134]:
geoData.head()

,person,region,city
0,4886f805,Rio de Janeiro,Cantagalo
1,ad93850f,Sao Paulo,São Paulo
2,0297fc1e,Rio de Janeiro,Niterói
3,2d681dd8,Sao Paulo,Juquitiba
4,cccea85e,Sao Paulo,Osasco


In [135]:
geoData= geoData.drop_duplicates(subset = "person", keep = "first")

In [136]:
geoData.head()

,person,region,city
0,4886f805,Rio de Janeiro,Cantagalo
1,ad93850f,Sao Paulo,São Paulo
2,0297fc1e,Rio de Janeiro,Niterói
3,2d681dd8,Sao Paulo,Juquitiba
4,cccea85e,Sao Paulo,Osasco


In [137]:
trainingSet["person"].count()

19414

In [138]:
geoData = pd.merge(geoData,trainingSet,how="right",on="person")

In [139]:
geoData["person"].count()

19414

In [140]:
geoData.head()

,person,region,city,label
0,ad93850f,Sao Paulo,São Paulo,0
1,1b9f7cf6,Unknown,Unknown,0
2,de8fe91b,Sao Paulo,São Paulo,0
3,45baf068,Unknown,Unknown,0
4,99abca5a,Minas Gerais,Betim,0


In [141]:
geoEncoding = geoData.drop("person",axis=1)
geoEncoding = geoEncoding.groupby(["region","city"],as_index=False).sum()

In [142]:
geoData = geoData.merge(geoEncoding, how= "left",on = ["region","city"])

In [143]:
geoData["person"].count()

19414

In [144]:
geoData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 5 columns):
person     19414 non-null object
region     19126 non-null object
city       19126 non-null object
label_x    19414 non-null int64
label_y    19126 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 910.0+ KB


In [145]:
geoData.head()

,person,region,city,label_x,label_y
0,ad93850f,Sao Paulo,São Paulo,0,132.0
1,1b9f7cf6,Unknown,Unknown,0,169.0
2,de8fe91b,Sao Paulo,São Paulo,0,132.0
3,45baf068,Unknown,Unknown,0,169.0
4,99abca5a,Minas Gerais,Betim,0,2.0


In [146]:
regionEncoding = geoData[["region","label_x"]]

In [147]:
regionEncoding.head()

,region,label_x
0,Sao Paulo,0
1,Unknown,0
2,Sao Paulo,0
3,Unknown,0
4,Minas Gerais,0


In [148]:
regionEncoding = regionEncoding.groupby("region",as_index=False).sum()

In [149]:
geoData = geoData.merge(regionEncoding,how = "left", on= "region")
geoData.head()

,person,region,city,label_x_x,label_y,label_x_y
0,ad93850f,Sao Paulo,São Paulo,0,132.0,285.0
1,1b9f7cf6,Unknown,Unknown,0,169.0,169.0
2,de8fe91b,Sao Paulo,São Paulo,0,132.0,285.0
3,45baf068,Unknown,Unknown,0,169.0,169.0
4,99abca5a,Minas Gerais,Betim,0,2.0,91.0


In [150]:
geoData["label_city_encoding"] = geoData["label_y"]
geoData["label_region_encoding"] = geoData["label_x_y"]

In [151]:
geoData.head()

,person,region,city,label_x_x,label_y,label_x_y,label_city_encoding,label_region_encoding
0,ad93850f,Sao Paulo,São Paulo,0,132.0,285.0,132.0,285.0
1,1b9f7cf6,Unknown,Unknown,0,169.0,169.0,169.0,169.0
2,de8fe91b,Sao Paulo,São Paulo,0,132.0,285.0,132.0,285.0
3,45baf068,Unknown,Unknown,0,169.0,169.0,169.0,169.0
4,99abca5a,Minas Gerais,Betim,0,2.0,91.0,2.0,91.0


In [152]:
geoDataMerge = geoData[["person","label_city_encoding","label_region_encoding"]]

In [153]:
geoDataMerge.head()

,person,label_city_encoding,label_region_encoding
0,ad93850f,132.0,285.0
1,1b9f7cf6,169.0,169.0
2,de8fe91b,132.0,285.0
3,45baf068,169.0,169.0
4,99abca5a,2.0,91.0


In [154]:
geoDataMerge["label_city_encoding"] = geoDataMerge["label_city_encoding"].fillna(0)
geoDataMerge["label_region_encoding"] = geoDataMerge["label_city_encoding"].fillna(0)
geoDataMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 3 columns):
person                   19414 non-null object
label_city_encoding      19414 non-null float64
label_region_encoding    19414 non-null float64
dtypes: float64(2), object(1)
memory usage: 606.7+ KB


/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [155]:
setTrain = pd.read_csv('setEntrenamiento.csv')

In [156]:
setTrain.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,label
0,0,0008ed71,3,3.0,0.0,0.0,0.0,1,0,1.0,...,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,1,000c79fe,1,1.0,0.0,1.0,0.0,1,8,0.0,...,17.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,0
2,2,001802e4,1,1.0,0.0,5.0,0.0,1,0,0.0,...,19.0,0.0,1.0,4.0,0.0,2.0,0.0,0.0,1.0,0
3,3,0019e639,21,15.0,2.0,29.0,0.0,1,3,1.0,...,202.0,1.0,3.0,89.0,110.0,2.0,24.0,0.0,0.0,0
4,4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0,0.0,...,7.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0


In [157]:
setTrain = pd.merge(setTrain,geoDataMerge,on = "person",how = "inner")

In [158]:
setTrain["person"].count()

19414

In [159]:
setTrain.to_csv('setEntrenamiento+labelencode.csv')

In [160]:
setTest = pd.read_csv("setTest.csv")

In [164]:
setTest.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,ventas_region,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0,0.0,...,0.0,9.0,0.0,1.0,4.0,0.0,4.0,0.0,0.0,1.0
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,5,0.0,...,0.0,126.0,0.0,1.0,96.0,2.0,130.0,0.0,86.0,0.0
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,1,1.0,...,0.0,26.0,0.0,1.0,13.0,4.0,1.0,0.0,0.0,1.0
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0,1.0,...,0.0,519.0,0.0,0.0,465.0,204.0,32.0,122.0,0.0,0.0
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,9,0.0,...,0.0,257.0,0.0,2.0,177.0,3.0,64.0,0.0,24.0,1.0


In [162]:
setTest = setTest.drop(["label_city_encoding","label_region_encoding"],axis=1)

In [163]:
setTest["person"].count()

19415

In [165]:
geoDataMergeTest = geoData[["region","city","label_city_encoding","label_region_encoding"]]

In [166]:
geoDataMergeTest.head()

,region,city,label_city_encoding,label_region_encoding
0,Sao Paulo,São Paulo,132.0,285.0
1,Unknown,Unknown,169.0,169.0
2,Sao Paulo,São Paulo,132.0,285.0
3,Unknown,Unknown,169.0,169.0
4,Minas Gerais,Betim,2.0,91.0


In [167]:
geoTest = eventsData[["person","region","city"]].drop_duplicates(subset = "person", keep = "first")

In [168]:
geoTest.head()

,person,region,city
0,4886f805,Rio de Janeiro,Cantagalo
1,ad93850f,Sao Paulo,São Paulo
2,0297fc1e,Rio de Janeiro,Niterói
3,2d681dd8,Sao Paulo,Juquitiba
4,cccea85e,Sao Paulo,Osasco


In [169]:
setTest = pd.merge(setTest,geoTest,how ="inner",on = "person")

In [170]:
setTest.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,region,city
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0,0.0,...,0.0,1.0,4.0,0.0,4.0,0.0,0.0,1.0,Rio de Janeiro,Cantagalo
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,5,0.0,...,0.0,1.0,96.0,2.0,130.0,0.0,86.0,0.0,Rio de Janeiro,Niterói
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,1,1.0,...,0.0,1.0,13.0,4.0,1.0,0.0,0.0,1.0,Sao Paulo,Juquitiba
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0,1.0,...,0.0,0.0,465.0,204.0,32.0,122.0,0.0,0.0,Sao Paulo,Osasco
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,9,0.0,...,0.0,2.0,177.0,3.0,64.0,0.0,24.0,1.0,Sao Paulo,Santos


In [171]:
setTest["person"].count()

19415

In [172]:
geoDataMergeTest = geoDataMergeTest.drop_duplicates(subset= "city", keep= "first" )

In [173]:
setTest = pd.merge(setTest, geoDataMergeTest,how = "left", on = "city")

In [174]:
setTest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 67 columns):
Unnamed: 0                      19415 non-null int64
person                          19415 non-null object
cant_seciones                   19415 non-null int64
event_checkout                  19415 non-null float64
event_conversion                19415 non-null float64
event_ad campaign hit           19415 non-null float64
event_lead                      19415 non-null float64
skus                            19415 non-null int64
Busca_terminos_populares        19415 non-null int64
Computer                        19415 non-null float64
Smartphone                      19415 non-null float64
otros                           19415 non-null float64
marca_apple                     19415 non-null float64
marca_asus                      19415 non-null float64
marca_lenovo                    19415 non-null float64
marca_lg                        19415 non-null float64
marca_motorola      

In [175]:
setTest["person"].count()

19415

In [176]:
setTest["label_city_encoding"] = setTest["label_city_encoding"].fillna(0)
setTest["label_region_encoding"] = setTest["label_region_encoding"].fillna(0)

In [177]:
setTest.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,region_x,city,region_y,label_city_encoding,label_region_encoding
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0,0.0,...,0.0,4.0,0.0,0.0,1.0,Rio de Janeiro,Cantagalo,NaN,0.0,0.0
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,5,0.0,...,2.0,130.0,0.0,86.0,0.0,Rio de Janeiro,Niterói,Rio de Janeiro,1.0,78.0
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,1,1.0,...,4.0,1.0,0.0,0.0,1.0,Sao Paulo,Juquitiba,Sao Paulo,0.0,285.0
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0,1.0,...,204.0,32.0,122.0,0.0,0.0,Sao Paulo,Osasco,Sao Paulo,11.0,285.0
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,9,0.0,...,3.0,64.0,0.0,24.0,1.0,Sao Paulo,Santos,Sao Paulo,2.0,285.0


In [178]:
setTest.columns

Index(['Unnamed: 0', 'person', 'cant_seciones', 'event_checkout',
       'event_conversion', 'event_ad campaign hit', 'event_lead', 'skus',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'channel_Direct',
    

In [179]:
setTest = setTest[[ 'person', 'cant_seciones', 'event_checkout',
       'event_conversion', 'event_ad campaign hit', 'event_lead', 'skus',
       'Busca_terminos_populares', 'Computer', 'Smartphone', 'otros',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'channel_Direct',
       'channel_Paid', 'channel_Organic', 'channel_Referral', 'Brasilero',
       'ventas_region', 'count_activity', 'conversiones_15lastdays',
       'checkout_15lastdays', 'viewed_15lastday', 'color_Preto',
       'color_Dourado', 'color_Branco', 'color_Cinza espacial',
       'new_15lastdays',  'label_city_encoding',
       'label_region_encoding']]

In [180]:
setTest.to_csv("setTest+labelenconde.csv")